[개념정리]
=============
to_datetime 메서드 : 문자열을 datetime 오브젝트로 변환해줌

시간 형식 지정자 - pg.245

strftime 메서드, 시간 형식 지정자를 이용해서 시계열 데이터 잘라내기

dt 접근자, quarter 속성(dt.quarter) 이용해서 은행 파산 분기 알아내기

shift 메서드 : 인잣값만큼 데이터를 밀어내는 메서드

reindex 메서드 사용해서 새로 생성한 인덱스(new_idx)를 새로운 인덱스로 지정

last_valid_index : 가장 오래된 데이터를 가져옴

first_valid_index : 가장 최근의 데이터를 가져옴

In [ ]:
# pg.243
import pandas as pd
import os
ebola = pd.read_csv('/content/sample_data/country_timeseries.csv')    # ebola 데이터 집합 불러옴

In [ ]:
print(ebola.info( ))    # Date열이 문자열로 저장되어 있음

In [ ]:
# date_dt라는 새로운 열에 Date열의 자료형을 datetime오브젝트로 변환한 결과 넣어줌
ebola['date_dt'] = pd.to_datetime(ebola['Date'])
print(ebola.info( ))

In [ ]:
# 시간 형식 지정자 (%d, %m, %y)
test_df1 = pd.DataFrame({'order_day':['01/01/15', '02/01/15', '03/01/15']})
test_df1['date_dt1'] = pd.to_datetime(test_df1['order_day'], format = '%d/%m/%y')
test_df1['date_dt2'] = pd.to_datetime(test_df1['order_day'], format = '%m/%d/%y')
test_df1['date_dt3'] = pd.to_datetime(test_df1['order_day'], format = '%y/%m/%d')
print(test_df1)

In [ ]:
test_df2 = pd.DataFrame({'order_day':['01-01-15', '02-01-15', '03-01-15']})
test_df2['date_dt'] = pd.to_datetime(test_df2['order_day'], format='%d-%m-%y')
print(test_df2)

In [ ]:
# pg.246 : 시계열 데이터를 구분해서 추출
# strftime 메서드, 시간 형식 지정자를 이용해서 시계열 데이터 잘라내기
from datetime import datetime
now = datetime.now()
print(now)

In [ ]:
nowDate = now.strftime('%Y-%m-%d')
print(nowDate)

In [ ]:
nowTime = now.strftime('%H:%M:%S')
print(nowTime)

In [ ]:
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(nowDatetime)

In [ ]:
# pg.247 : datetime 오브젝트로 변환하려는 열을 지정하여 데이터 집합 불러오기
# parse_dates 인자에 datetime 오브젝트로 변환하고자 하는 열의 이름 전달해서 데이터 집합 불러오기
ebolal = pd.read_csv('/content/sample_data/country_timeseries.csv', parse_dates=['Date']) 
print(ebolal.info( ))

In [ ]:
# datetime 오브젝트에서 날짜 정보 추출하기
# 문자열로 저장된 날짜를 시리즈에 담아 datetime 오브젝트로 변환함
date_series = pd.Series(['2018-05-16', '2018-05-17', '2018-05-18'])
d1 = pd.to_datetime(date_series)
print(d1)

In [ ]:
print(d1[0].year)
print(d1[0].month)
print(d1[0].day)

In [ ]:
# pg.248 : dt접근자로 시계열 데이터 정리하기
ebola = pd.read_csv('/content/sample_data/country_timeseries.csv')
ebola['date_dt'] = pd.to_datetime(ebola['Date'])

In [ ]:
print(ebola[['Date', 'date_dt']].head())

In [ ]:
print(ebola['date_dt'][3].year)

In [ ]:
print(ebola['date_dt'][3].month)

In [ ]:
print(ebola['date_dt'][3].day)

In [ ]:
# dt접근자로 date_df열에 한번에 접근 -> year 속성 이용해서 연도값을 추출
ebola['year'] = ebola['date_dt'].dt.year
print(ebola[['Date', 'date_dt', 'year']].head( ))

In [ ]:
ebola['month'], ebola['day'] = (ebola['date_dt'].dt.month, ebola['date_dt'].dt.day)
print(ebola[['Date', 'date_dt', 'year', 'month', 'day']].head( ))

In [ ]:
print(ebola.info())   # date_dt열은 datetime오브젝트 / 나머지는 정수형

In [ ]:
# pg.251 : 에볼라 최초 발병일 계산하기
print(ebola.iloc[-5:,:5])   # ebola 데이터프레임의 마지막 행,열 5개 보기

In [ ]:
print(ebola['date_dt'].min( ))    # 에볼라가 발생하기 시작한 날
print(type(ebola['date_dt'].min( )))

In [ ]:
ebola['outbreak_d'] = ebola['date_dt'] - ebola['date_dt'].min()   # 에볼라의 진행 정도
print(ebola[['Date','Day','outbreak_d']].head())

In [ ]:
# pg.252 : 파산한 은행의 개수 계산하기
banks = pd.read_csv('/content/sample_data/banklist.csv')
print(banks.head())   # Closing Date / Updated Date 열의 데이터 자료형이 시계열 데이터임

In [ ]:
banks_no_dates = pd.read_csv('/content/sample_data/banklist.csv')
print(banks_no_dates.info())

In [ ]:
banks = pd.read_csv('/content/sample_data/banklist.csv', parse_dates=[5,6])   # datetime 오브젝트로 변환해서 불러옴
print(banks.info())

In [ ]:
# dt 접근자, quarter 속성 이용해서 은행 파산 분기 알아내기
banks['closing_quarter'], banks['closing_year'] = (banks['Closing Date'].dt.quarter, banks['Closing Date'].dt.year)
print(banks.head())

In [ ]:
# groupby 메서드 이용해서 연도별로 파산한 은행의 개수 알아내기
closing_year = banks.groupby(['closing_year']).size()
print(closing_year)

In [ ]:
# 각 연도별, 분기별로 파산한 은행의 개수 알아보기
closing_year_q = banks.groupby(['closing_year', 'closing_quarter']).size()
print(closing_year_q)

In [ ]:
# 연도별 , 분기별 파산 은행개수 그래프 그리기
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
ax = closing_year.plot()
plt.show()

fig,ax = plt.subplots()
ax = closing_year_q.plot()
plt.show()

In [ ]:
# pg.264 : 에볼라의 확산 속도 비교하기
import matplotlib.pyplot as plt

ebola.index = ebola['Date']

fig, ax = plt.subplots( )
ax = ebola.iloc[0:, 1:20].plot(ax=ax)    # 1: 으로 하니깐 너무 커서 임의로 20으로 설정
ax.legend(fontsize=5, loc=2, borderaxespad=0.)
plt.show()    

각 나라의 에볼라 발병일이 달라서 그래프가 그려지기 시작한 지점도 다름

각 나라의 발병일을 가장 처음에 에볼라가 발병한 Guinea와 동일한 위치로 옮겨야 나라별 에볼라의 확산속도를 제대로 비교할 수 있음

In [ ]:
ebola_sub = ebola[['Day','Cases_Guinea','Cases_Liberia']]
print(ebola_sub.tail(10))

In [ ]:
ebola = pd.read_csv('/content/sample_data/country_timeseries.csv', parse_dates=['Date'])
print(ebola.head().iloc[:,:5])

In [ ]:
print(ebola.tail().iloc[:,:5])

In [ ]:
ebola.index = ebola['Date']   # Date열을 인덱스로 지정
new_idx = pd.date_range(ebola.index.min(), ebola.index.max())   # 날짜가 아예 없었던 데이터의 인덱스를 생성가능케 함

In [ ]:
print(new_idx)

In [ ]:
new_idx = reversed(new_idx)   # reversed 메서드 사용해서 인덱스를 반대로 뒤집음

In [ ]:
ebola = ebola.reindex(new_idx)    # reindex 메서드 사용해서 새로 생성한 인덱스(new_idx)를 새로운 인덱스로 지정
print(ebola.head().iloc[:,:5])

In [ ]:
# last_valid_index : 가장 오래된 데이터를 가져옴
last_valid = ebola.apply(pd.Series.last_valid_index)
print(last_valid)

In [ ]:
# first_valid_index : 가장 최근의 데이터를 가져옴
first_valid = ebola.apply(pd.Series.first_valid_index) 
print(first_valid)

In [ ]:
earliest_date = ebola.index.min( ) 
print(earliest_date)

In [ ]:
shift_values = last_valid - earliest_date   # (에볼라 처음 발병날 - 각 나라의 에볼라 발병일) 만큼 옮기기
print(shift_values)

In [ ]:
ebola_dict = {}
for idx, col in enumerate(ebola):
  d = shift_values[idx].days
  shifted = ebola[col].shift(d)   # shift 메서드 : 인잣값만큼 데이터를 밀어내는 메서드
  ebola_dict[col] = shifted

In [ ]:
ebola_shift = pd.DataFrame(ebola_dict)    # 딕셔너리를 데이터프레임으로 변환
print(ebola_shift.tail())

In [ ]:
print(ebola_shift.tail())   # 에볼라의 최초 발병일 기준으로 모든 열의 데이터가 옮겨져있음

In [ ]:
ebola_shift.index = ebola_shift['Day']    # 인덱스를 Day 열로 지정
ebola_shift = ebola_shift.drop(['Date', 'Day'], axis=1)   # 그래프에 필요없는 Date, Day 열 삭제
print(ebola_shift.tail( ))

In [ ]:
fig, ax = plt.subplots( ) 
ax = ebola_shift.iloc[:, :].plot(ax=ax) 
ax.legend(fontsize=7, loc=2, borderaxespad=0.)
plt.show( )